In [20]:
import pandas as pd
import numpy as np
import pathlib
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
import os
import argparse
import pyfaidx
import sys
import re
import scipy

from helper_functions import *
# bamdir = "/mnt/archiving/DATA_HIEUNHO/GW-bismark_cfDNA-highdepth_pair-end/04_bismark_deduplicated_sorted"
# region = "1:1103264-1103363"
# path_to_all_fa = "/datassd/hieunguyen/hg19"
# metadata = pd.read_csv("/mnt/DATASM14/hieunho/hieu_project/metadata/ECD/metadata_GW_highdepth.csv")
# bamfiles = [item for item in pathlib.Path(bamdir).glob("*.bam") if str(item) in metadata.Bam_file.unique()]

# print("Number of bam files in this location: {}".format(len(bamfiles)))

# outputdir = "./outputs"
# os.system("mkdir -p {}".format(outputdir))

# betadf = generate_betadf(bamfiles = bamfiles, 
#                          region = region, 
#                          path_to_all_fa = path_to_all_fa, 
#                          outputdir = outputdir, 
#                          save_file =  True)

metadata = pd.read_csv("metadata_GW_highdepth.csv", index_col = [0])
betadf = pd.read_csv("./outputs/1_1103264_1103363_beta_values.all_samples.csv", index_col = [0])

betadf["label"] = betadf["sample"].apply(lambda x: metadata[metadata["Filename"] == x]["Label"].unique()[0])

subset_metadata = dict()
for sampletype in metadata.Sample_type.unique():
    subset_metadata[sampletype] = list(metadata[metadata["Sample_type"] == sampletype].Filename.unique())
    
# filter: keep tissue and wbc samples only. 
subset_betadf = betadf[betadf["sample"].isin(subset_metadata["Tissue"] + subset_metadata["WBC"])]
resdf = pd.DataFrame()
for group in [item for item in metadata.Label.unique() if item != "Control"]:
    target_group = subset_betadf[subset_betadf["label"] == group].avg_beta.to_numpy()
    background_group = subset_betadf[subset_betadf["label"] != group].avg_beta.to_numpy()
    target_group = [item for item in target_group if np.isnan(item) == False]
    background_group = [item for item in background_group if np.isnan(item) == False]
    pval = scipy.stats.ttest_ind(target_group, 
                            background_group).pvalue
    logFC = np.log2(0.0001 + (np.mean(target_group)/np.mean(background_group))) # add 0.0001 to avoid log(0)
    res = {"group": group, "pval": pval, "logFC": logFC}
    tmpdf = pd.DataFrame.from_dict(res, orient = "index").T

    resdf = pd.concat([resdf, tmpdf], axis = 0)